# Modeling Cineplex Concessions - Simple Version

- Team Bloor
- November 2020
- MMAI 869

This Notebook will build a basic ML classifier model. Please feel free to use and modify as you see fit.

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
os.getcwd()

'/content'

# Load in the data 

In [3]:
version_str = "_v3"
input_dir = "clean_v3/"
gen_dir = 'gen_v3/'

In [4]:
visits = pd.read_csv(input_dir+"visits"+version_str+".csv")
locations = pd.read_csv(input_dir+"locations"+version_str+".csv", encoding = "ISO-8859-1")
items = pd.read_csv(input_dir+"items"+version_str+".csv", encoding = "ISO-8859-1")
visits_items = pd.read_csv(input_dir+"visit_items"+version_str+".csv", encoding = "ISO-8859-1")
films = pd.read_csv(input_dir+"films"+version_str+".csv", encoding = "ISO-8859-1")
experience_type = pd.read_csv(input_dir+"experience_type"+version_str+".csv", encoding = "ISO-8859-1")
sales_channels = pd.read_csv(input_dir+"sales_channels"+version_str+".csv", encoding = "ISO-8859-1")
timeslice = pd.read_csv(input_dir+"timeslice"+version_str+".csv", encoding = "ISO-8859-1")

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(visits.head())

,Visit_ID,CINEPLEX_D_Membership_ID,Visit_Date,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Spend,Revenue,Points_Earned,Points_Redeemed,Discount,Has_US_Is_LrgPopcorn,Has_US_Is_Butter,Has_US_Is_Alcohol,Has_US_Is_Candy,Has_US_Is_Coffee,Has_US_Is_Kiddietray,Has_US_Is_Voucher,Has_US_Is_MenuFood
0,2,23777,2019-01-01,15,7253,1,28757,1,8,1,1,15.0,15.0,100,0,1.7,0,0,0,0,0,0,0,0
1,3,60723,2019-01-01,15,7123,0,0,0,8,0,0,29.6,29.6,297,0,3.3,0,1,0,0,0,0,1,1
2,4,61080,2019-01-01,15,7290,2,24899,2,8,2,1,28.4,28.4,235,0,3.2,1,1,0,0,0,0,0,0
3,5,103538,2019-01-01,14,1142,1,24977,1,2,1,1,0.0,10.0,0,1000,0.0,0,0,0,0,0,0,0,0
4,6,120196,2019-01-01,23,9181,1,29694,1,2,1,1,17.6,17.6,100,0,1.9,0,0,0,0,0,0,0,0


# Basic Wrangling and Cleaning

- Merge all the fact tables (e.g., `films`, `locations`) into on big `visits` table - this table will act as the "historical truth"
- Fill in missing values with "Unknown"
- Extract some date info (e.g., like day of week, name of month, etc)


In [6]:
visits.shape

visits = visits.merge(films, how="left", on="Film_ID")
visits = visits.merge(experience_type, how="left")
visits = visits.merge(sales_channels, how="left")
visits = visits.merge(timeslice, left_on="Visit_TimeSlice_ID", right_on="TimeSlice_ID", how="left")
visits = visits.merge(locations, how="left")

visits.shape

(36258, 24)

(36258, 60)

In [7]:
# Fill in empty values on language
visits['Market_Language'] = visits['Market_Language'].fillna('Unknown')

In [8]:
# Extract some info about the date
visits['Visit_Date'] = pd.to_datetime(visits['Visit_Date'])
visits['Visit_Date_DOW'] = visits['Visit_Date'].dt.day_name()
visits['Visit_Date_Week'] = visits['Visit_Date'].dt.strftime('%U')
visits['Visit_Date_Month'] = visits['Visit_Date'].dt.month_name()
visits['Visit_Date_IsWeekend'] = np.where(visits['Visit_Date'].dt.weekday < 5, 0, 1)

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(visits.head())

,Visit_ID,CINEPLEX_D_Membership_ID,Visit_Date,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Spend,Revenue,Points_Earned,Points_Redeemed,Discount,Has_US_Is_LrgPopcorn,Has_US_Is_Butter,Has_US_Is_Alcohol,Has_US_Is_Candy,Has_US_Is_Coffee,Has_US_Is_Kiddietray,Has_US_Is_Voucher,Has_US_Is_MenuFood,Film_Title,Title_ID,Title_Name,Theatrical_Release_Date,Film_Runtime,Language,Market_Language,Release_Pattern,Performance_Type,Is_IMAX,Is_DBOX_Capable,Release_Status,Media_Class_Description,Is_4Dx,Genre_Hierarchy_1,Auditorium_Experience_Description,Is_Premium,Sales_Channel,TimeSlice_ID,TimeSlice_Description,TimeSlice_Start_Time,TimeSlice_End_Time,TimeSlice_Hour_12,TimeSlice_Hour_24,Location_Name,Location_Short_Name,Location_Interactive_Name,Address1,City,Province_CD,Postal_Code,Country_CD,Is_VIP,Is_Outtakes,Is_Licensed_Bar,Is_Licensed_Lounge,Visit_Date_DOW,Visit_Date_Week,Visit_Date_Month,Visit_Date_IsWeekend
0,2,23777,2019-01-01,15,7253,1,28757,1,8,1,1,15.0,15.0,100,0,1.7,0,0,0,0,0,0,0,0,Vice,23956.0,Vice,12/25/2018,133.0,English,English,Wide,Film Presentation,0,0,A,Adult,0.0,Drama,Regular,0,Point of Sale,15,1 - Early Matinee,00:00.0,30:00.0,1:00 PM,13,CPX Eglinton T.C.,Warden&Eglin,Cineplex Odeon Eglinton Town Centre Cinemas,22 Lebovic Avenue,Toronto,ON,M1L 4V9,CA,0,1,1,0,Tuesday,00,January,0
1,3,60723,2019-01-01,15,7123,0,0,0,8,0,0,29.6,29.6,297,0,3.3,0,1,0,0,0,0,1,1,<None>,0.0,<None>,1/1/1900,0.0,<None>,<None>,<None>,<None>,0,0,B,None,0.0,Unknown,<Unknown>,0,Point of Sale,15,1 - Early Matinee,00:00.0,30:00.0,1:00 PM,13,CPX Winston Churchill VIP,Winston Churchill,Cineplex Cinemas Winston Churchill & VIP,2081 Winston Park Dr.,Oakville,ON,L6H 6P5,CA,1,1,1,0,Tuesday,00,January,0
2,4,61080,2019-01-01,15,7290,2,24899,2,8,2,1,28.4,28.4,235,0,3.2,1,1,0,0,0,0,0,0,Aquaman,22133.0,Aquaman,12/21/2018,143.0,English,English,Wide,Film Presentation,0,1,A,Adult,0.0,Adaptation,AVX,1,Point of Sale,15,1 - Early Matinee,00:00.0,30:00.0,1:00 PM,13,CPX Hamilton Mountain,CPX Hamilton,Cineplex Cinemas Hamilton Mountain,795 Paramount Drive,Stoney Creek,ON,L8J 0B4,CA,0,1,1,0,Tuesday,00,January,0
3,5,103538,2019-01-01,14,1142,1,24977,1,2,1,1,0.0,10.0,0,1000,0.0,0,0,0,0,0,0,0,0,Mary Poppins Returns,22187.0,Mary Poppins Returns,12/19/2018,131.0,English,English,Wide,Film Presentation,0,1,A,Family,0.0,Comedy,Regular,0,Kiosk,14,1 - Early Matinee,30:00.0,00:00.0,12:00 PM,12,CPX Pitt Meadows,Meadowtown,Cineplex Odeon Meadowtown Cinemas,"#410, 19800 Lougheed Highway",Pitt Meadows,BC,V3Y 2W1,CA,0,1,0,0,Tuesday,00,January,0
4,6,120196,2019-01-01,23,9181,1,29694,1,2,1,1,17.6,17.6,100,0,1.9,0,0,0,0,0,0,0,0,DeuxiÂme acte,23976.0,Second Act,12/21/2018,104.0,French,French,Wide,Film Presentation,0,0,A,Adult,0.0,Comedy,Regular,0,Kiosk,23,2 - Late Matinee,00:00.0,30:00.0,5:00 PM,17,CPX Beauport,Beauport,CinÂma Cineplex Odeon Beauport,825 rue ClÂmenceau,Beauport,QC,G1C 2K6,CA,0,0,0,0,Tuesday,00,January,0


In [10]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36258 entries, 0 to 36257
Data columns (total 64 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Visit_ID                           36258 non-null  int64         
 1   CINEPLEX_D_Membership_ID           36258 non-null  int64         
 2   Visit_Date                         36258 non-null  datetime64[ns]
 3   Visit_TimeSlice_ID                 36258 non-null  int64         
 4   Location_ID                        36258 non-null  int64         
 5   Auditorium_Experience_ID           36258 non-null  int64         
 6   Film_ID                            36258 non-null  int64         
 7   Seating_Experience_ID              36258 non-null  int64         
 8   Sales_Channel_ID                   36258 non-null  int64         
 9   Performance_Experience_ID          36258 non-null  int64         
 10  Performance_Type_ID               

In [11]:
# `X` will act as the main training data. 
# Think of `df` as "what will be available during prediction time" and `visits` as `historical information from which we can engineer
# features. 
# This `df` will be the same as visits, except with columsn removed that either (a) would not be available
X = visits.copy()


remove_cols = [

            # Can't have these (won't be available in future)
           'Spend', 'Revenue', 'Points_Earned', 'Points_Redeemed', 'Discount',
           'Has_US_Is_LrgPopcorn', 'Has_US_Is_Butter', 'Has_US_Is_Alcohol', 'Has_US_Is_Candy',
           'Has_US_Is_Coffee', 'Has_US_Is_Kiddietray', 'Has_US_Is_Voucher', 'Has_US_Is_MenuFood',
    
            # Don't want these (won't be helpful, I don't think.)
           'TimeSlice_Description', 'TimeSlice_Start_Time', 'TimeSlice_End_Time','TimeSlice_Hour_12', 'TimeSlice_Hour_24',
           'Film_Title', 'Title_Name', 'Theatrical_Release_Date', 'Release_Status', 'Release_Pattern', 'Title_ID', 
           'Location_Name', 'Location_Short_Name', 'Location_Interactive_Name',
           'Address1', 'City', 'Postal_Code',
          ] 

X = X.drop(remove_cols, axis=1)


In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36258 entries, 0 to 36257
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Visit_ID                           36258 non-null  int64         
 1   CINEPLEX_D_Membership_ID           36258 non-null  int64         
 2   Visit_Date                         36258 non-null  datetime64[ns]
 3   Visit_TimeSlice_ID                 36258 non-null  int64         
 4   Location_ID                        36258 non-null  int64         
 5   Auditorium_Experience_ID           36258 non-null  int64         
 6   Film_ID                            36258 non-null  int64         
 7   Seating_Experience_ID              36258 non-null  int64         
 8   Sales_Channel_ID                   36258 non-null  int64         
 9   Performance_Experience_ID          36258 non-null  int64         
 10  Performance_Type_ID               

# Training/Test Split

In [13]:
# The name of the column in visits that has the target value we want to predict
# Change this to whatever you need/want!
target_col = 'Has_US_Is_MenuFood'

y = visits[target_col]

train_index = X['Visit_ID'] <= 88863
test_index = ~train_index


visits_train, visits_test = visits[train_index], visits[test_index]

visits_train.shape
visits_test.shape

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

X_train.shape
y_train.shape

X_test.shape
y_test.shape

(31079, 64)

(5179, 64)

(31079, 34)

(31079,)

(5179, 34)

(5179,)

# Feature Engineering (On the Training Data Only)

We need to take special care avoid data leakage. In particular, when we are engineering features that involves calculations of the data, those calculations should be trained (fit) from the training data.  

## Per-Member Aggregate Features

We now calcualte member-level aggregation statistics, such as:
 - The number of total visits
 - The number of differnet locations the member has visited
 - The number of times a member has purchased large popcorn
 - The number of times a member has purchased butter
 - etc.

In [14]:
# Define my own version of the 'mode' function, which will just return the first mode (in case of tie) and 
# thus avoid an annoying error from groupby (whcih always wants just one value)
my_mode = lambda x: pd.Series.mode(x)[0]

# Use a simple group by to compute some aggregrate statistics. We can do anything we want here,
# this is just an example!
members_features = visits_train.groupby('CINEPLEX_D_Membership_ID').agg(
{
    'Visit_Date': ['count'],

    'Film_ID': [pd.Series.nunique, my_mode],
    'Film_Runtime': ['mean', 'max'],
    'Is_VIP': ['mean', 'max'],
    'Market_Language': [my_mode],
    'Genre_Hierarchy_1': [pd.Series.nunique, my_mode],

    'Spend': ['min', 'mean', 'max', 'sum'],
    'Discount': ['mean'],

    'Has_US_Is_LrgPopcorn': ['sum', 'mean', 'max'],
    'Has_US_Is_Butter': ['sum', 'mean', 'max'],
    'Has_US_Is_Alcohol': ['sum', 'mean', 'max'],
    'Has_US_Is_Candy': ['sum', 'mean', 'max'],
    'Has_US_Is_Kiddietray': ['sum', 'mean', 'max'],
    'Has_US_Is_Voucher': ['sum', 'mean', 'max'],
    'Has_US_Is_MenuFood': ['sum', 'mean', 'max'],
    'Has_US_Is_Coffee': ['sum', 'mean', 'max'],
}).reset_index()


# Pandas group-by creates a MultiIndex, which we don't want. The following few lines
# will rename the columns of the dataframe to something more reasonbale.
members_features.columns = ["_".join(x) for x in members_features.columns.ravel()]
members_features = members_features.rename(columns={'CINEPLEX_D_Membership_ID_': "CINEPLEX_D_Membership_ID"})
members_features.columns = members_features.columns.str.replace("<lambda_0>", "mode")
members_features.columns = members_features.columns.str.replace("<lambda>", "mode")

# We want to make sure we have all numeric data, so let's do OHE (dummies) to be sure.
members_features = pd.get_dummies(members_features)

In [15]:
# Let's take a peek
members_features.shape
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    members_features.head()

(5778, 56)

,CINEPLEX_D_Membership_ID,Visit_Date_count,Film_ID_nunique,Film_ID_mode,Film_Runtime_mean,Film_Runtime_max,Is_VIP_mean,Is_VIP_max,Genre_Hierarchy_1_nunique,Spend_min,Spend_mean,Spend_max,Spend_sum,Discount_mean,Has_US_Is_LrgPopcorn_sum,Has_US_Is_LrgPopcorn_mean,Has_US_Is_LrgPopcorn_max,Has_US_Is_Butter_sum,Has_US_Is_Butter_mean,Has_US_Is_Butter_max,Has_US_Is_Alcohol_sum,Has_US_Is_Alcohol_mean,Has_US_Is_Alcohol_max,Has_US_Is_Candy_sum,Has_US_Is_Candy_mean,Has_US_Is_Candy_max,Has_US_Is_Kiddietray_sum,Has_US_Is_Kiddietray_mean,Has_US_Is_Kiddietray_max,Has_US_Is_Voucher_sum,Has_US_Is_Voucher_mean,Has_US_Is_Voucher_max,Has_US_Is_MenuFood_sum,Has_US_Is_MenuFood_mean,Has_US_Is_MenuFood_max,Has_US_Is_Coffee_sum,Has_US_Is_Coffee_mean,Has_US_Is_Coffee_max,Market_Language_mode_<None>,Market_Language_mode_English,Market_Language_mode_French,Genre_Hierarchy_1_mode_Action,Genre_Hierarchy_1_mode_Adaptation,Genre_Hierarchy_1_mode_Adventure,Genre_Hierarchy_1_mode_Comedy,Genre_Hierarchy_1_mode_Documentary,Genre_Hierarchy_1_mode_Drama,Genre_Hierarchy_1_mode_Family,Genre_Hierarchy_1_mode_Fantasy,Genre_Hierarchy_1_mode_Foreign,Genre_Hierarchy_1_mode_Horror,Genre_Hierarchy_1_mode_Musical,Genre_Hierarchy_1_mode_Sequel,Genre_Hierarchy_1_mode_Suspense,Genre_Hierarchy_1_mode_Thriller,Genre_Hierarchy_1_mode_Unknown
0,3752,2,2,0,46.5,93.0,0.0,0,2,0.0,6.3,12.7,12.7,0.7,0,0.0,0,1,0.5,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,5052,2,2,27465,89.0,90.0,0.0,0,1,54.2,69.2,84.2,138.3,7.7,0,0.0,0,0,0.0,0,0,0.0,0,2,1.0,1,0,0.0,0,0,0.0,0,1,0.5,1,0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,5548,21,20,0,107.5,149.0,0.0,0,7,0.0,14.3,38.9,300.1,1.6,6,0.3,1,0,0.0,0,0,0.0,0,1,0.0,1,0,0.0,0,0,0.0,0,2,0.1,1,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,7157,6,6,0,98.7,143.0,0.0,0,4,7.5,21.2,38.8,126.9,2.4,4,0.7,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15934,1,1,28848,103.0,103.0,0.0,0,1,18.9,18.9,18.9,18.9,2.1,1,1.0,1,1,1.0,1,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    visits_train[visits_train['CINEPLEX_D_Membership_ID']==7157]

,Visit_ID,CINEPLEX_D_Membership_ID,Visit_Date,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Spend,Revenue,Points_Earned,Points_Redeemed,Discount,Has_US_Is_LrgPopcorn,Has_US_Is_Butter,Has_US_Is_Alcohol,Has_US_Is_Candy,Has_US_Is_Coffee,Has_US_Is_Kiddietray,Has_US_Is_Voucher,Has_US_Is_MenuFood,Film_Title,Title_ID,Title_Name,Theatrical_Release_Date,Film_Runtime,Language,Market_Language,Release_Pattern,Performance_Type,Is_IMAX,Is_DBOX_Capable,Release_Status,Media_Class_Description,Is_4Dx,Genre_Hierarchy_1,Auditorium_Experience_Description,Is_Premium,Sales_Channel,TimeSlice_ID,TimeSlice_Description,TimeSlice_Start_Time,TimeSlice_End_Time,TimeSlice_Hour_12,TimeSlice_Hour_24,Location_Name,Location_Short_Name,Location_Interactive_Name,Address1,City,Province_CD,Postal_Code,Country_CD,Is_VIP,Is_Outtakes,Is_Licensed_Bar,Is_Licensed_Lounge,Visit_Date_DOW,Visit_Date_Week,Visit_Date_Month,Visit_Date_IsWeekend
439,762,7157,2019-01-04,27,7268,2,24899,1,12,2,1,13.2,29.2,132,3000,1.5,1,0,0,0,0,0,0,0,Aquaman,22133.0,Aquaman,12/21/2018,143.0,English,English,Wide,Film Presentation,0,1,A,Adult,0.0,Adaptation,AVX,1,Cineplex Mobile App,27,3 - Early Evening,00:00.0,30:00.0,7:00 PM,19,GLX Waterloo,Conestoga,Galaxy Cinemas Waterloo,550 King Street North,Waterloo,ON,N2L 5W6,CA,0,1,1,0,Friday,00,January,0
1139,2018,7157,2019-01-10,28,7268,1,23805,1,12,2,1,38.5,38.5,150,0,4.3,0,0,0,0,0,0,0,0,Spider-Man: Into The Spider-Verse,21425.0,Spider-Man: Into The Spider-Verse,12/14/2018,117.0,English,English,Wide,Film Presentation,0,1,A,Family,0.0,Adaptation,Regular,0,Cineplex Mobile App,28,3 - Early Evening,30:00.0,00:00.0,7:00 PM,19,GLX Waterloo,Conestoga,Galaxy Cinemas Waterloo,550 King Street North,Waterloo,ON,N2L 5W6,CA,0,1,1,0,Thursday,01,January,0
5152,11943,7157,2019-03-01,28,7268,1,22496,1,12,2,1,38.8,38.8,282,0,4.3,1,0,0,0,0,0,0,0,The Lego Movie 2: The Second Part,20450.0,The Lego Movie 2: The Second Part,2/8/2019,107.0,English,English,Wide,Film Presentation,0,1,A,Family,0.0,Comedy,Regular,0,Cineplex Mobile App,28,3 - Early Evening,30:00.0,00:00.0,7:00 PM,19,GLX Waterloo,Conestoga,Galaxy Cinemas Waterloo,550 King Street North,Waterloo,ON,N2L 5W6,CA,0,1,1,0,Friday,08,March,0
16663,35489,7157,2019-06-13,26,7269,1,24975,1,12,1,1,10.6,21.4,125,1250,1.2,0,0,0,0,0,0,0,0,Aladdin,22185.0,Aladdin,5/24/2019,128.0,English,English,Wide,Film Presentation,0,1,A,Family,0.0,Fantasy,Regular,0,Cineplex Mobile App,26,3 - Early Evening,30:00.0,00:00.0,6:00 PM,18,GLX Cambridge,G Cambridge,Galaxy Cinemas Cambridge,355 Hespeler Road,Cambridge,ON,N1R 6B3,CA,0,0,1,0,Thursday,23,June,0
27735,56141,7157,2019-09-25,28,7268,0,0,0,8,0,0,7.5,7.5,37,0,0.8,1,0,0,0,0,0,0,0,<None>,0.0,<None>,1/1/1900,0.0,<None>,<None>,<None>,<None>,0,0,B,None,0.0,Unknown,<Unknown>,0,Point of Sale,28,3 - Early Evening,30:00.0,00:00.0,7:00 PM,19,GLX Waterloo,Conestoga,Galaxy Cinemas Waterloo,550 King Street North,Waterloo,ON,N2L 5W6,CA,0,1,1,0,Wednesday,38,September,0
29835,74584,7157,2019-10-18,29,7268,1,29558,4,12,1,1,18.3,29.1,163,1250,2.0,1,0,0,0,0,0,0,0,Zombieland 2: Double Tap,24481.0,Zombieland 2: Double Tap,10/18/2019,97.0,English,English,Wide,Film Presentation,0,1,A,Adult,1.0,Comedy,Regular,0,Cineplex Mobile App,29,3 - Early Evening,00:00.0,30:00.0,8:00 PM,20,GLX Waterloo,Conestoga,Galaxy Cinemas Waterloo,550 King Street North,Waterloo,ON,N2L 5W6,CA,0,1,1,0,Friday,41,October,0


## One Hot Encoding of Categorical Features

In [17]:
from sklearn.preprocessing import OneHotEncoder

# Which columns are categorical (that we want to encode)?
cat_feature_names = ['Language', 'Market_Language', 'Performance_Type',
       'Media_Class_Description', 'Visit_Date_DOW', 'Visit_Date_Month',
       'Genre_Hierarchy_1', 'Auditorium_Experience_Description',
       'Sales_Channel', 'Province_CD', 'Country_CD']
print(cat_feature_names)

# Fit the OneHotEncoder and save the `ohe` object for later.
ohe = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
ohe = ohe.fit(X_train[cat_feature_names])
ohe_feature_names = list(ohe.get_feature_names(cat_feature_names))

print("\n\nNames of {} OHE features:".format(len(ohe_feature_names)))
print(ohe_feature_names)

['Language', 'Market_Language', 'Performance_Type', 'Media_Class_Description', 'Visit_Date_DOW', 'Visit_Date_Month', 'Genre_Hierarchy_1', 'Auditorium_Experience_Description', 'Sales_Channel', 'Province_CD', 'Country_CD']


Names of 103 OHE features:
['Language_<None>', 'Language_Arabic', 'Language_Cantonese', 'Language_Egyptian', 'Language_English', 'Language_Filipino', 'Language_French', 'Language_German', 'Language_Gujarati', 'Language_Hebrew', 'Language_Hindi', 'Language_Italian', 'Language_Japanese', 'Language_Korean', 'Language_Lebanese', 'Language_Mandarin', 'Language_Polish', 'Language_Punjabi', 'Language_Spanish', 'Language_Telugu', 'Language_Thai', 'Language_Urdu', 'Language_Vietnamese', 'Market_Language_<None>', 'Market_Language_English', 'Market_Language_French', 'Performance_Type_<None>', 'Performance_Type_Alt Prog Film Event', 'Performance_Type_Concert', 'Performance_Type_Documentary', 'Performance_Type_FRCE Feature Release', 'Performance_Type_Film Presentation', 'Performa

## Main Function to Do Feature Transformations

Now that the feature engieering steps have been trained/fit, we can now extract features from the data.

The following function will take raw data (whehter it be training data or test dat) and do the following:
- Encode categorical features (using the OHE trained above)
- Look up the member's statistics from previous visits. (If member has not previous visits, averages/means are used.)
- Remove columns that aren't needed.

In [18]:
def get_features(X):
    # This function assumes that the following variables have already been assigned:
    # - cat_attrs and cat_feature_names
    # - members_features
    
    # Use the OHE to get numerical features from categorical
    ohe_features = ohe.transform(X[cat_feature_names])
    ohe_features_df = pd.DataFrame(ohe_features, columns=ohe_feature_names)

    # Look up member aggregation statistics
    X_features = X.merge(members_features, how="left", on="CINEPLEX_D_Membership_ID" ).reset_index()
    
    # For any members that didn't have agg features, then just fill them in with mean values.
    X_features = X_features.fillna(members_features.mean())
    
    # Remove features we don't want any more
    X_features = X_features.drop(cat_feature_names, axis=1)
    X_features = X_features.drop(['Visit_ID', 'Visit_Date', 'index'], axis=1)
    
    # Merge the numeric features and encoded features together.
    features_all = pd.concat([X_features, ohe_features_df], axis=1)

    
    return features_all

### Feature Selection

In [19]:
# X_train_features = get_features(X_train)

# from sklearn.feature_selection import VarianceThreshold
# sel = VarianceThreshold(threshold=(0.1));
# sel.fit(X_train_features);

# sel.variances_
# sel.get_support()
# sum(sel.get_support())

### OverSampling

In [20]:
# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=0)

X_train_features = get_features(X_train)

# X_resampled, y_resampled = ros.fit_resample(X_train_features, y_train)
# X_resampled.shape
# y_resampled.shape
# np.bincount(y_resampled)

from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train_features, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

# #evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over Random'))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(55212, 179)

(55212,)

array([27606, 27606])

### UnderSampling

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

# X_train_features = get_features(X_train)

# rus = RandomUnderSampler(random_state=0)
# X_resampled, y_resampled = rus.fit_resample(X_train_features, y_train)

# X_resampled.shape
# y_resampled.shape
# np.bincount(y_resampled)

### Training

In [21]:
#X_train_features = get_features(X_train)
#X_train_new = X_train_features.loc[:, sel.get_support()]

# Let's take a peek
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    X_resampled.head()

AttributeError: ignored

### Testing

In [22]:
X_test_features = get_features(X_test)
#X_test_new = X_test_features.loc[:, sel.get_support()]

# Let's take a peek
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    X_test_features.head()
    
    X_test_features.info()

,CINEPLEX_D_Membership_ID,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Film_Runtime,Is_IMAX,Is_DBOX_Capable,Is_4Dx,Is_Premium,TimeSlice_ID,Is_VIP,Is_Outtakes,Is_Licensed_Bar,Is_Licensed_Lounge,Visit_Date_Week,Visit_Date_IsWeekend,Visit_Date_count,Film_ID_nunique,Film_ID_mode,Film_Runtime_mean,Film_Runtime_max,Is_VIP_mean,Is_VIP_max,Genre_Hierarchy_1_nunique,Spend_min,Spend_mean,Spend_max,Spend_sum,Discount_mean,Has_US_Is_LrgPopcorn_sum,Has_US_Is_LrgPopcorn_mean,Has_US_Is_LrgPopcorn_max,Has_US_Is_Butter_sum,Has_US_Is_Butter_mean,Has_US_Is_Butter_max,Has_US_Is_Alcohol_sum,Has_US_Is_Alcohol_mean,Has_US_Is_Alcohol_max,Has_US_Is_Candy_sum,Has_US_Is_Candy_mean,Has_US_Is_Candy_max,Has_US_Is_Kiddietray_sum,Has_US_Is_Kiddietray_mean,Has_US_Is_Kiddietray_max,Has_US_Is_Voucher_sum,Has_US_Is_Voucher_mean,Has_US_Is_Voucher_max,Has_US_Is_MenuFood_sum,Has_US_Is_MenuFood_mean,Has_US_Is_MenuFood_max,Has_US_Is_Coffee_sum,Has_US_Is_Coffee_mean,Has_US_Is_Coffee_max,Market_Language_mode_<None>,Market_Language_mode_English,Market_Language_mode_French,Genre_Hierarchy_1_mode_Action,Genre_Hierarchy_1_mode_Adaptation,Genre_Hierarchy_1_mode_Adventure,Genre_Hierarchy_1_mode_Comedy,Genre_Hierarchy_1_mode_Documentary,Genre_Hierarchy_1_mode_Drama,Genre_Hierarchy_1_mode_Family,Genre_Hierarchy_1_mode_Fantasy,Genre_Hierarchy_1_mode_Foreign,Genre_Hierarchy_1_mode_Horror,Genre_Hierarchy_1_mode_Musical,Genre_Hierarchy_1_mode_Sequel,Genre_Hierarchy_1_mode_Suspense,Genre_Hierarchy_1_mode_Thriller,Genre_Hierarchy_1_mode_Unknown,Language_<None>,Language_Arabic,Language_Cantonese,Language_Egyptian,Language_English,Language_Filipino,Language_French,Language_German,Language_Gujarati,Language_Hebrew,Language_Hindi,Language_Italian,Language_Japanese,Language_Korean,Language_Lebanese,Language_Mandarin,Language_Polish,Language_Punjabi,Language_Spanish,Language_Telugu,Language_Thai,Language_Urdu,Language_Vietnamese,Market_Language_<None>,Market_Language_English,Market_Language_French,Performance_Type_<None>,Performance_Type_Alt Prog Film Event,Performance_Type_Concert,Performance_Type_Documentary,Performance_Type_FRCE Feature Release,Performance_Type_Film Presentation,Performance_Type_Opera,Performance_Type_Rec Room Event,Performance_Type_Speaking Engagement,Performance_Type_Sporting Event,Performance_Type_Stage Performance,Performance_Type_Television Event,Media_Class_Description_Adult,Media_Class_Description_Family,Media_Class_Description_None,Visit_Date_DOW_Friday,Visit_Date_DOW_Monday,Visit_Date_DOW_Saturday,Visit_Date_DOW_Sunday,Visit_Date_DOW_Thursday,Visit_Date_DOW_Tuesday,Visit_Date_DOW_Wednesday,Visit_Date_Month_April,Visit_Date_Month_August,Visit_Date_Month_February,Visit_Date_Month_January,Visit_Date_Month_July,Visit_Date_Month_June,Visit_Date_Month_March,Visit_Date_Month_May,Visit_Date_Month_November,Visit_Date_Month_October,Visit_Date_Month_September,Genre_Hierarchy_1_Action,Genre_Hierarchy_1_Adaptation,Genre_Hierarchy_1_Adventure,Genre_Hierarchy_1_Animated,Genre_Hierarchy_1_Comedy,Genre_Hierarchy_1_Documentary,Genre_Hierarchy_1_Drama,Genre_Hierarchy_1_Family,Genre_Hierarchy_1_Fantasy,Genre_Hierarchy_1_Foreign,Genre_Hierarchy_1_Horror,Genre_Hierarchy_1_Musical,Genre_Hierarchy_1_Period,Genre_Hierarchy_1_SciFi,Genre_Hierarchy_1_Sequel,Genre_Hierarchy_1_Suspense,Genre_Hierarchy_1_Thriller,Genre_Hierarchy_1_Unknown,Auditorium_Experience_Description_4DX,Auditorium_Experience_Description_<Unknown>,Auditorium_Experience_Description_AVX,Auditorium_Experience_Description_Alt.Prog Event,Auditorium_Experience_Description_Alt.Prog Film,Auditorium_Experience_Description_IMAX,Auditorium_Experience_Description_Regular,Auditorium_Experience_Description_ScreenX,Auditorium_Experience_Description_VIP,Sales_Channel_Cineplex Mobile App,Sales_Channel_Kiosk,Sales_Channel_Mobile Web Browser,Sales_Channel_Online (Cineplex.com),Sales_Channel_Other,Sales_Channel_Point of Sale,Province_C

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5179 entries, 0 to 5178
Columns: 179 entries, CINEPLEX_D_Membership_ID to Country_CD_CA
dtypes: float64(160), int64(16), object(3)
memory usage: 7.1+ MB


# (Optional) Write the training/testing files out

For inspection, or use in other programs/tools

In [ ]:
# Want to include all 8 targets in the output files
target_cols = ['Has_US_Is_LrgPopcorn', 'Has_US_Is_Butter', 'Has_US_Is_Alcohol','Has_US_Is_Candy',
    'Has_US_Is_Kiddietray','Has_US_Is_Voucher','Has_US_Is_MenuFood','Has_US_Is_Coffee']

y_all = visits[target_cols]

y_all_train, y_all_test = y_all[train_index], y_all[test_index]

X_train_features.to_csv(gen_dir+'gen_X_train_features'+version_str+'.csv', index=False)
y_all_train.to_csv(gen_dir+'gen_y_train'+version_str+'.csv', index=False)
X_test_features.to_csv(gen_dir+'gen_X_test_features'+version_str+'.csv', index=False)
y_all_test.to_csv(gen_dir+'gen_y_test'+version_str+'.csv', index=False)

# Machine Learning Time! (Modeling)

### Hyperparameter Tuning and Training

In [ ]:
# from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import BaggingClassifier

# clf_bag = BaggingClassifier(
#     DecisionTreeClassifier(max_depth=None, min_samples_split=2), max_samples = 0.1, max_features=0.5, random_state=0)
# tscv = TimeSeriesSplit(n_splits=5)

# params = {'max_depth': [None, 10, 30], 
#               'n_estimators': [1000,1200,1500], 
#               'criterion':['gini', 'entropy']}

# gridsearch = GridSearchCV(clf_bag, params, scoring='f1_macro', cv=tscv, return_train_score=True, n_jobs = 10)

# %time gridsearch.fit(X_resampled, y_resampled)

## Training

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf_gt = GradientBoostingClassifier(
    n_estimators = 1300, learning_rate=0.1, max_depth=2, max_features=3, random_state=0)

%time clf_gt.fit(X_resampled, y_resampled)

CPU times: user 30.3 s, sys: 299 ms, total: 30.6 s
Wall time: 30.6 s


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=2,
                           max_features=3, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1300,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Testing

In [24]:
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, f1_score, log_loss, confusion_matrix

y_pred = clf_gt.predict(X_test_features)

# How did we do?
print("F1 Score   = {:.3f}".format(f1_score(y_test, y_pred, average="macro")))
print("Accuracy   = {:.3f}".format(accuracy_score(y_test, y_pred)))
print("Kappa      = {:.3f}".format(cohen_kappa_score(y_test, y_pred)))
print("Log Loss   = {:.3f}".format(log_loss(y_test, y_pred)))
print("\nConfusion Matrix:")
unique_label = np.unique([y_test, y_pred])
cmtx = pd.DataFrame(
confusion_matrix(y_test, y_pred, labels=unique_label), 
index=['true:{:}'.format(x) for x in unique_label], 
columns=['pred:{:}'.format(x) for x in unique_label])
print(cmtx)
print("\nClassification Report:")
print(classification_report(y_test, y_pred)) 

F1 Score   = 0.676
Accuracy   = 0.866
Kappa      = 0.353
Log Loss   = 4.635

Confusion Matrix:
        pred:0  pred:1
true:0    4223     368
true:1     327     261

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      4591
           1       0.41      0.44      0.43       588

    accuracy                           0.87      5179
   macro avg       0.67      0.68      0.68      5179
weighted avg       0.87      0.87      0.87      5179



## Model Inference

What did our model "learn"? What features are most important, etc?

In [27]:
# Let's measure the importances of the features
feature_names = X_train_features.columns

imp = pd.DataFrame({'Feature': feature_names, 'Importance': clf_gt.feature_importances_})
imp = imp.sort_values('Importance', ascending=False)
imp.head(10)

,Feature,Importance
53,Has_US_Is_MenuFood_mean,0.190864
54,Has_US_Is_MenuFood_max,0.139043
52,Has_US_Is_MenuFood_sum,0.108827
167,Sales_Channel_Point of Sale,0.039187
159,Auditorium_Experience_Description_Regular,0.022531
119,Visit_Date_DOW_Saturday,0.020524
44,Has_US_Is_Candy_mean,0.020362
141,Genre_Hierarchy_1_Drama,0.019405
20,Visit_Date_IsWeekend,0.016084
30,Spend_mean,0.015718
